# Mean Soil Adjusted Vegetation Index 2 Calculation

The modified soil-adjusted vegetation index (MSAVI) and its later revision, MSAVI2, are soil adjusted vegetation indices that seek to address some of the limitation of NDVI when applied to areas with a high degree of exposed soil surface. The problem with the original soil-adjusted vegetation index (SAVI) is that it required specifying the soil-brightness correction factor (L) through trial-and-error based on the amount of vegetation in the study area. Not only did this lead to the majority of people just using the default L value of 0.5, but it also created a circular logic problem of needing to know what the vegetation amount/cover was before you could apply SAVI which was supposed to give you information on how much vegetation there was developed the MSAVI, and later the MSAVI2 to more reliably and simply calculate a soil brightness correction factor.

MSAVI2 = (2 * NIR + 1 – sqrt ((2 * NIR + 1)2 – 8 * (NIR - R))) / 2

Here we have used Sentinel-2 data. It is a wide-swath, high-resolution, multi-spectral imaging mission supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. Band 8 and Band 4 used and NIR and RED bands respectively along with QA60 band for cloud masking. Required MSAVI2 formula for Sentinel-2 will be:
MSAVI = (2 * Band 8 + 1 – sqrt ((2 * Band 8 + 1)2 – 8 * (Band 8 – Band 4))) / 2.

'geometry1' is the place 'Lakha' from Punjab. We can see the decrease in MSAVI 2 values from month of January to June. This could be possible because of wheat crop cultivation in initial months and harvesting in later. 'geometry2' is the place from Bihar where also we can se similar trends. 'geometry3' is Malkajgiri, Telangana area where we can observe different trend as compared to previous to regions. Here, MSAVI2 vsalues are kind of constant on linear curve fitting scale.

Results and time series graphs with detailed code is shown below:

We need to add ee python library to access earth engine in python and ipygee to plot graphs from the output values

In [1]:
from ipygee import *

import ee
ee.Initialize() #To initialize google earth engine servers in our program

Here we are creating polygon for the given 3 places.

In [2]:
geometry1 = ee.Geometry.Polygon(
        [[[75.45157492160797, 30.634404129904425],
          [75.4524278640747, 30.63372099804389],
          [75.45236885547638, 30.634399514164798],
          [75.45157492160797, 30.634404129904425]]])
geometry2 = ee.Geometry.Polygon(
        [[[85.85622847080823, 26.09425078918021],
          [85.8559066057052, 26.093581136401006],
          [85.8568647212981, 26.094009907326967],
          [85.85622847080823, 26.09425078918021]]])
geometry3 = ee.Geometry.Polygon(
        [[[78.66571158170699, 17.66869116558751],
          [78.6662346124649, 17.6686911655875],
          [78.6662346124649, 17.66929686130703],
          [78.66571158170699, 17.66869116558751]]])

Function for the MSAVI 2 calculation has been created in next step

In [3]:
def addmsavi2(image):
    msavi2 = image.expression(
      '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
      {
        'NIR': image.select('B8'),
        'RED': image.select('B4')
      }
    )
    return image.addBands(msavi2.rename('msavi2'))  #adding all the image as a seperate band in the data set


Function to remove opaque clouds and cirrus clouds

QA60 is band for cloud mask

Bitmask for QA60:
    Bit 10: Opaque clouds
        0: No opaque clouds
        1: Opaque clouds present
    Bit 11: Cirrus clouds
        0: No cirrus clouds
        1: Cirrus clouds present

In [4]:
def maskcloud1(image):
    QA60 = image.select(['QA60'])
    return image.updateMask(QA60.lt(1))   #both bit 10's and 11's cloud present is removed


We will import satellite image collection from Jan 2020 to June 2020, filter area as per given geometry. After that cloud masking and adding MSAVI 2 layer added.

In [5]:
S2 = ee.ImageCollection("COPERNICUS/S2").filterDate('2020-01-01', '2020-06-30').filterBounds(geometry1)

S2 = S2.map(maskcloud1)    

S2 = S2.map(addmsavi2)

Creating time series chart using ipygee library of python by msavi band data which has been created before

In [6]:
plot_msavi2_1 = chart.Image.series(**{
    'imageCollection': S2, 
    'region': geometry1,
    'bands': ['msavi2'],
    'reducer': ee.Reducer.mean(),
    'scale': 500,
    'label_properties':['MSAVI2'],
    'title': 'MSAVI2 time series'})

plot_msavi2_1.renderWidget(width='75%')    #render the charts

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

Similar plot for the 2nd geometry

In [7]:
S2 = ee.ImageCollection("COPERNICUS/S2").filterDate('2020-01-01', '2020-06-30').filterBounds(geometry2)

S2 = S2.map(maskcloud1)

S2 = S2.map(addmsavi2)

plot_msavi2_2 = chart.Image.series(**{
    'imageCollection': S2, 
    'region': geometry2,
    'bands': ['msavi2'],
    'reducer': ee.Reducer.mean(),
    'scale': 500,
    'xProperty': 'system:time_start',
    'title': 'MSAVI2 time series'})
plot_msavi2_2.renderWidget(width='75%')


HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

Similar plot for the 3rd geometry

In [8]:
S2 = ee.ImageCollection("COPERNICUS/S2").filterDate('2020-01-01', '2020-06-30').filterBounds(geometry3)

S2 = S2.map(maskcloud1)

S2 = S2.map(addmsavi2)

plot_msavi2_3 = chart.Image.series(**{
    'imageCollection': S2, 
    'region': geometry3,
    'bands': ['msavi2'],
    'reducer': ee.Reducer.mean(),
    'scale': 500,
    'xProperty': 'system:time_start',
    'title': 'MSAVI2 time series'})
plot_msavi2_3.renderWidget(width='75%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…